In [ ]:
import tensorflow as tf
print(tf.__version__)


In [ ]:
import sys
print(sys.version)


In [ ]:
pip install tensorflow opencv-python streamlit numpy matplotlib

In [ ]:
from tensorflow.keras.models import load_model
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Step 1: Load the pretrained model
model = load_model("../model/face_mask_detector.model")


# Step 2: Load and display a sample test image
img = cv2.imread("test_face.jpeg")  # Add any photo of a face and name it test_face.jpg
print("Image loaded:", img is not None)

img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img_rgb)
plt.axis('off')
plt.title("Original Image")
plt.show()

# Step 3: Preprocess the image for the model
img_resized = cv2.resize(img, (224, 224))
img_normalized = img_resized / 255.0
img_input = np.expand_dims(img_normalized, axis=0)  # Add batch dimension

# Step 4: Get prediction
prediction = model.predict(img_input)[0][0]

# Step 5: Interpret the prediction
if prediction > 0.5:
    print("🟢 MASK detected ✅ (Confidence:", prediction, ")")
else:
    print("🔴 NO MASK detected ❌ (Confidence:", prediction, ")")


In [ ]:
# Step 6: Add label to the image
label = "MASK" if prediction > 0.5 else "NO MASK"
color = (0, 255, 0) if prediction > 0.5 else (255, 0, 0)

# Add label text to image (on a copy to preserve original)
img_copy = img.copy()
cv2.putText(img_copy, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 
            1.0, color, 2, cv2.LINE_AA)

# Convert to RGB for matplotlib
img_rgb_labeled = cv2.cvtColor(img_copy, cv2.COLOR_BGR2RGB)

# Display with matplotlib
plt.figure(figsize=(6, 6))
plt.imshow(img_rgb_labeled)
plt.axis('off')
plt.title(f"Prediction: {label}")
plt.show()

# Save the result
cv2.imwrite("predicted_test_face.png", img_copy)
print("✅ Image saved as 'predicted_test_face.png'")


In [ ]:
# Load OpenCV's pre-trained Haar cascade face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Convert image to grayscale (required for Haar cascades)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [ ]:
# Detect faces → returns a list of rectangles (x, y, w, h)
faces = face_cascade.detectMultiScale(gray, scaleFactor=1.05, minNeighbors=3, minSize=(30, 30))

print("Faces found:", len(faces))

# Draw rectangles around faces and predict for each
for (x, y, w, h) in faces:
    # Crop the face region
    face = img[y:y+h, x:x+w]

    # Resize, normalize, and expand dimensions
    face_resized = cv2.resize(face, (224, 224)) / 255.0
    face_input = np.expand_dims(face_resized, axis=0)

    # Predict
    pred = model.predict(face_input)[0][0]
    label = "MASK ✅" if pred > 0.5 else "NO MASK ❌"
    confidence = round(pred if pred > 0.5 else 1 - pred, 3)

    # Draw bounding box + label
    color = (0, 255, 0) if pred > 0.5 else (0, 0, 255)
    cv2.rectangle(img, (x, y), (x+w, y+h), color, 2)
    cv2.putText(img, f"{label} ({confidence})", (x, y - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

# Show the image with predictions
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title("Face Detection + Prediction")
plt.show()

In [ ]:
# Load a new image
img2 = cv2.imread("test_face_2.jpg")
gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

# Detect face
faces2 = face_cascade.detectMultiScale(gray2, scaleFactor=1.05, minNeighbors=3, minSize=(30, 30))
print("Faces found:", len(faces2))

# Predict on each face
for (x, y, w, h) in faces2:
    face = img2[y:y+h, x:x+w]
    face_resized = cv2.resize(face, (224, 224)) / 255.0
    face_input = np.expand_dims(face_resized, axis=0)
    pred = model.predict(face_input)[0][0]
    label = "MASK ✅" if pred > 0.5 else "NO MASK ❌"
    confidence = round(pred if pred > 0.5 else 1 - pred, 3)
    color = (0, 255, 0) if pred > 0.5 else (0, 0, 255)
    
    cv2.rectangle(img2, (x, y), (x+w, y+h), color, 2)
    cv2.putText(img2, f"{label} ({confidence})", (x, y - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

# Show result
plt.imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.title("Second Face Test")
plt.show()
